In [65]:
import pandas as pd
from time import mktime, strptime
import datetime

In [88]:
DATABASE_ADDRESS = "../Data/Raw/chartevents.csv"
PROCEDURE_EVENTS_ADDRESS = "../Data/Raw/procedureevents.csv"
CODE_MAPPING_ADDRESS = "../Data/Raw/code_mappings.csv"
DATA_SIZE = 100*1000000

###### 20大概需要14分鐘
###### 1需要30.5MB
###### 大檔案有 329,499789行

In [67]:
FILENAME = datetime.datetime.now().strftime('%Y%m%d') + f'_{DATA_SIZE:.0e}'
FILENAME

'20230706_2e+07'

In [68]:
item = dict()
mark = tuple()

In [69]:
df = pd.read_csv(CODE_MAPPING_ADDRESS, engine="python", encoding="unicode_escape")
id_to_label = dict(zip(df["itemid"], df["label"]))
label_to_id = dict(zip(df["label"], df["itemid"]))
# df.head()

In [70]:
mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius", \
        "Temperature Fahrenheit")

In [89]:
%%time 
df = pd.read_csv(DATABASE_ADDRESS, usecols=["hadm_id", "charttime", "itemid", "value"], engine="python", encoding="unicode_escape", nrows=DATA_SIZE)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   hadm_id    int64  
 1   charttime  object 
 2   itemid     int64  
 3   value      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 61.0+ MB
CPU times: user 6.35 s, sys: 341 ms, total: 6.69 s
Wall time: 6.94 s


In [72]:
df['charttime'] = pd.to_datetime(df['charttime']).round("10min")

In [73]:
# 建立一個CLASS容器來裝每一個PATIENT
class Patient:
    data = dict()
    def __init__(self, ):
        self.data = {"subject_id":None, "hadm_id":None, "charttime":None}
        for label_ in mark:
            self.data[label_] = None
    def allna(self, ):
        return all ([self.data[i]==None for i in mark])

In [74]:
output = []
cnt = 0
tp = [label_to_id[i] for i in mark]
for name, group in df.groupby(["hadm_id", "charttime"]):
    patient = Patient()
    for id_, value_ in zip(group.itemid, group.value):
        cnt += 1
        if id_ in tp:
            patient.data[id_to_label[id_]] = value_
    if not patient.allna():
    # if True:
        patient.data["hadm_id"], patient.data["charttime"] = name
        if patient.data["Temperature Fahrenheit"] != None:
            patient.data["Temperature Celsius"] = round((float(patient.data["Temperature Fahrenheit"]) - 32) * 5 / 9, 1)
        output.append(patient.data)
merged_df = pd.DataFrame(output,columns=['hadm_id','charttime']+list(mark)).drop(["Temperature Fahrenheit"], axis=1)
merged_df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_pivot.csv", float_format='%.2f', index=0)

In [75]:
# df.to_csv("chart_test3_merge_new.csv", float_format='%.2f', index=0)
print("amount: ", len(output))
print("amount: ", len(merged_df))
# merged_df

amount:  923360
amount:  923360


In [76]:
# ------------------------TODO------------------------ #

In [77]:
testdf = merged_df

In [78]:
testdf.sample()

,hadm_id,charttime,Heart Rate,Respiratory Rate,Non Invasive Blood Pressure systolic,Non Invasive Blood Pressure diastolic,O2 saturation pulseoxymetry,Temperature Celsius
130406,21588174,2166-06-19 15:00:00,70,23,None,None,94,None


In [79]:
new_mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius")
testdf[2:3][list(new_mark)]

,Heart Rate,Respiratory Rate,Non Invasive Blood Pressure systolic,Non Invasive Blood Pressure diastolic,O2 saturation pulseoxymetry,Temperature Celsius
2,95,18,None,None,99,None


In [80]:
df = pd.read_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_pivot.csv", engine="python", encoding="unicode_escape", parse_dates=["charttime"])
df.dropna(thresh=5, inplace=True)
for name, group in df.groupby("hadm_id"):
    group.interpolate(method='bfill', inplace=True)
    group.interpolate(method='ffill', inplace=True)
    df.update(group)
df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_filled.csv", float_format='%.1f', index=0)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 827399 entries, 0 to 923359
Data columns (total 8 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   hadm_id                                827399 non-null  int64         
 1   charttime                              827399 non-null  datetime64[ns]
 2   Heart Rate                             827399 non-null  float64       
 3   Respiratory Rate                       827355 non-null  float64       
 4   Non Invasive Blood Pressure systolic   821804 non-null  float64       
 5   Non Invasive Blood Pressure diastolic  821784 non-null  float64       
 6   O2 saturation pulseoxymetry            827360 non-null  float64       
 7   Temperature Celsius                    826287 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 56.8 MB


In [82]:
def read_csv_function(road: str) -> pd.core.frame.DataFrame:
    print("Reading", road, "...")
    return pd.read_csv(road, engine='python')
def to_timestamp(s: str):
    try:
        return int(mktime(strptime(s, "%Y-%m-%d %H:%M:%S")))
    except ValueError:
        try:
            return int(mktime(strptime(s, "%Y-%m-%d %H:%M")))
        except ValueError:
            return int(mktime(strptime(s, "%Y/%m/%d %H:%M")))

In [83]:
df = read_csv_function(PROCEDURE_EVENTS_ADDRESS)
dic = {}
for i, it in enumerate(df.itemid):
    if it != 225466:
        continue
    if df.iloc[i].hadm_id not in dic:
        dic[df.iloc[i].hadm_id] = list()
    dic[df.iloc[i].hadm_id].append(to_timestamp(str(df.iloc[i].starttime)))

df = read_csv_function(f"../Data/Preprocessed/chartevents_{FILENAME}_filled.csv")
ans = [False]*len(df)
for i in range(len(df)):
    if int(df.iloc[i].hadm_id) in dic:
        for j in dic[df.iloc[i].hadm_id]:
            if 0 <= j - to_timestamp(str(df.iloc[i].charttime)) <= 14400:
                ans[i] = True
                break

df["label"] = ans
df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_labeled.csv", float_format='%.2f', index=0)

Reading ../Data/Raw/procedureevents.csv ...
Reading ../Data/Preprocessed/chartevents_20230706_2e+07_filled.csv ...


In [84]:
df.label.value_counts()

label
False    827148
True        251
Name: count, dtype: int64